In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import *
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from torchvision.io import read_image
import os
from torchvision.utils import save_image
import numpy as np
import torch.nn.functional as F 

In [103]:
annotations_file = 'indexeddata.csv'

In [104]:
class Ja3far(Dataset):
    def __init__(self,annotations_file, transform=None, target_transform=None):
        self.img_path_label = pd.read_csv(annotations_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_path_label)

    def __getitem__(self, idx):
        img_path = self.img_path_label.iloc[idx, 1]
        image = read_image(img_path)
        label = torch.tensor(self.img_path_label.iloc[idx, 2])
        obj = (image, img_path)
        if self.transform:
            obj = self.transform(obj)
        if self.target_transform:
            label = self.target_transform(label)
        return obj[0], label

In [111]:
class GrayScale:
    def __call__(self, obj):
        gray_img = Grayscale()(obj[0])
        gray_img = Resize((140, 140))(gray_img)
        return gray_img, obj[1]
class EditResolution:
    def __call__(self, obj):
        img = obj[0]
        path = obj[1]
        if(path[-3:] == 'png'):
            img = Resize((640, 640))(img)
        else:
            img = Resize((1280, 1280))(img)
        return img, path
class RotateImage:
    def __call__(self, obj):
        rotater = RandomRotation(degrees=(0, 180))
        rotated_img = rotater(obj[0])
        return rotated_img, obj[1]
class ConvertDtype:
    def __call__(self, label):
        label = label.to(torch.float)
        return label

In [128]:
dataset = Ja3far(annotations_file,
                 transform=transforms.Compose([EditResolution(), RotateImage(), GrayScale(),]),
                 target_transform=transforms.Compose([ConvertDtype(),]))
train_size, test_size = 0.7, 0.3
training_data, test_data = torch.utils.data.random_split(dataset, [train_size, test_size])

In [136]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=True)

In [143]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(1, 12, 3)
        self.conv2 = nn.Conv2d(12, 24, 3)
        self.conv3 = nn.Conv2d(24, 36, 3)
        self.l1 = nn.Linear(15 * 15 * 36, 200)
        self.l2 = nn.Linear(200, 1)

    def forward(self, x):
        #x = self.flatten(x)
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.sigmoid(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
model = NeuralNetwork()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=[0.9, 0.999])

In [144]:
def train_one_epoch(epoch_index):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(train_dataloader):
        # Every data instance is an input + label pair
        inputs, labels = data
        inputs = inputs.to(torch.float)
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        # Make predictions for this batch
        outputs = model(inputs).squeeze()
        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 10 == 0:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            #tb_x = epoch_index * len(train_dataloader) + i + 1
            #tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss
epoches = 5
for i in range(epoches):
     train_one_epoch(i)

  batch 1 loss: 0.0006931471824645996
  batch 11 loss: 0.006931471824645996
  batch 1 loss: 0.0006931471824645996
  batch 11 loss: 0.006931471824645996
  batch 1 loss: 0.0006931471824645996
  batch 11 loss: 0.006931471824645996
  batch 1 loss: 0.0006931471824645996
  batch 11 loss: 0.006931471824645996
  batch 1 loss: 0.0006931471824645996
  batch 11 loss: 0.006931471824645996


In [90]:
torch.save(model, 'model.pth')

In [145]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_dataloader:
        images, labels = data
        outputs = model(images.to(torch.float))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the {total} test images: {100 * correct // total} %')

Accuracy of the network on the 49 test images: 59 %


In [91]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv1): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(12, 36, kernel_size=(3, 3), stride=(1, 1))
  (l1): Linear(in_features=121104, out_features=400, bias=True)
  (l2): Linear(in_features=400, out_features=1, bias=True)
)
